In [ ]:
from datetime import datetime
import os
import pandas as pd


today = datetime.today()#.strftime("%d-%m-%Y")
# cur_dir = f"outputs/{today}"
# os.makedirs(cur_dir)


today.strftime("%Y-%m-%dT%H:%M")

In [ ]:
from utils import get_links

df = pd.read_csv("./outputs/2025-03-04T18:07/df_terms.csv")

df = df.set_index(["language", "term", "platform"]).sort_index()


In [ ]:
from tqdm import tqdm
from selenium import webdriver

import os
os.environ['MOZ_HEADLESS'] = '1'
num_links = 2


with webdriver.Firefox() as driver:
    for i, (cur_lang, cur_term, cur_platform) in enumerate(tqdm(df.index.drop_duplicates())):
        # cur_sub = df.loc[(cur_lang, cur_term, cur_platform)]
        # links_scraped = df.loc[(cur_lang, cur_term, cur_platform), "link"].str.len() > 1
        # print(links_scraped.sum())
        # if links_scraped.all():
        #     print(f"row {i} done, skipping", flush=True, end="...\n")
        #     continue 
    
        links = get_links(driver=driver, platform=cur_platform, term=cur_term, n=5)
        links = [l[:11] for l in links]

        links = links + [None]*num_links
        df.loc[(cur_lang, cur_term, cur_platform), "link"] = links[:num_links]
        df.loc[(cur_lang, cur_term, cur_platform), "done"] = True


---

In [ ]:
meta = pd.read_csv("./outputs/2025-03-04T18:07/metadata.csv")


In [ ]:
import yt_dlp as youtube_dl


def live_hook(d):
#     print("\n\n\n", d.keys(), d["duration"], "\n\n\n")
#     print(d["playlist"])
#     print(d["playlist_index"])
    
    if d["duration"] > 600:
        return f"duration > 600 (is {d['duration']})"
        
    if d["is_live"]:
        return "is_live"
    return None


ydl_opts = {
            'quiet': True,
            'verbose': False,
            "writesubtitles": True,
            # "subtitleslangs": "en",
            # "subtitlesformat": "srt",
            "writethumbnail": False, #True,

            "download_archive": "downloaded.txt",

            "ignoreerrors": True,
            "outtmpl": "videos/%(id)s.%(ext)s",
        
            # "format": "best",
            "socket_timeout": 1,
            "match_filter": live_hook
        }

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download(f"https://www.youtube.com/watch?v=QMaWNQi18Zw")


In [ ]:
import pandas as pd
from glob import glob

lawki_dir = "./outputs/test2/"

meta = pd.read_csv(f"{lawki_dir}/metadata.csv", lineterminator="\n").set_index("link")
with open(f"{lawki_dir}/downloaded.txt") as handle:
    dwnld = [l.split()[1] for l in handle.readlines()]
vids = glob(f"{lawki_dir}/videos/*")

meta2 = meta.loc[dwnld]


meta2.index, meta.index
[l for l in meta.index if any(l in f for f in vids)] 

meta.to_csv

In [ ]:
[l.replace(".txt", "") for l in 
"ar.txt  bn.txt  de.txt  en.txt  es.txt  fr.txt  hi.txt  id.txt  jp.txt  original.txt  pt.txt  ru.txt  ur.txt  zh-tw.txt".split()
]

langs = 'ar','de','es','fr','hi','id','ja','pt','ru','ur','zh-CN'

len(langs)

12*7

In [ ]:
import pandas as pd

pd.read_csv("outputs/full_test/terms.csv").shape

parameters:
 - 12 langs
 - 7 terms
 - 5 links

- scrape links: 14 minutes

In [ ]:
pd.read_csv("outputs/dry_wood/metadata.csv").duration.hist()

242/60

In [ ]:
import pandas as pd
from glob import glob

lawki_dir = "./outputs/c"

meta = pd.read_csv(f"{lawki_dir}/metadata.csv", lineterminator="\n").set_index("link")
with open(f"{lawki_dir}/downloaded.txt") as handle:
    dwnld = [l.split()[1] for l in handle.readlines()]
vids = glob(f"{lawki_dir}/videos/*")
vids = pd.Series(vids).apply(lambda s: s.split("/")[-1].split(".")[0])

# meta2 = meta.loc[dwnld]

len(meta.index), len(dwnld), len(vids),\
len(meta.index.intersection(dwnld)), len(meta.index.intersection(vids)),\
len(set(dwnld) & set(vids))


---

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

v1 = VideoFileClip("./outputs/soft_macho/videos/0C08yBpMiCI.webm").subclip(1, 4)
v2 = VideoFileClip("./outputs/soft_macho/videos/91jaWWyjnY0.mkv").subclip(1, 4)

final = concatenate_videoclips([v1, v2], method='compose').set_fps(24)

print(f"{final.duration=}, {final.fps=}")
final.write_videofile(f"test_merged3.mp4") 

---

In [1]:
from parameters import *
from rw import NeighbourRW
from glob import glob
from moviepy.editor import VideoFileClip, concatenate_videoclips
import pandas as pd


def file_from_link(files, link):
    matches = [f for f in files if link in f]
    if not matches:
        return None
        # raise ValueError(f"{link} not in {files}!")
    return matches[0]

lawki_dir = "./outputs/redstar"

meta = pd.read_csv(f"{lawki_dir}/metadata.csv").set_index("link")
space = pd.read_csv(f"{lawki_dir}/embeddings_paraphrase-multilingual-mpnet-base-v2.csv").set_index("link")
space = space.loc[meta.index].values


rw = NeighbourRW(n_neighbours=2, space=space, meta=meta)

steps = []
for _ in range(3):
    try:
        i, cur_row, (start, duration) = rw.step(0)
        steps.append([cur_row.name, start, duration])
    except ValueError:
        print("DONE!")
        break

steps = pd.DataFrame(steps, columns=["link", "start", "duration"])

print(f"{steps.duration.sum()=}", flush=True)


n_neighbours=4
steps.duration.sum()=29.7


In [2]:
video_files = glob(f"{lawki_dir}/videos/*")

clips = [(file_from_link(video_files, l), s, d)
            for i, (l, s, d) in steps.iterrows()]
# clips = [VideoFileClip(l).subclip(s, s+d)
             # for l, s, d in clips if l]

clips

[('./outputs/redstar/videos/YM_x4E7vTbo.webm', 6.6, 10.41),
 ('./outputs/redstar/videos/8R1BIIfx8V8.webm', 12.9, 9.77),
 ('./outputs/redstar/videos/PSh-rU2mh6k.webm', 15.3, 9.52)]

In [3]:
vs = [VideoFileClip(l) for l, s, d in clips if l]

In [4]:
[v.duration for v in vs], meta.loc[steps.link].duration

([22.57, 44.03, 52.03],
 link
 YM_x4E7vTbo    23.0
 8R1BIIfx8V8    44.0
 PSh-rU2mh6k    52.0
 Name: duration, dtype: float64)

In [5]:
concat = concatenate_videoclips(
    [v.subclip(s, s+d) for v, (l, s, d) in zip(vs, clips)]
, method='compose')

In [6]:
concat.write_videofile(f"test.mp4")

Moviepy - Building video test.mp4.
MoviePy - Writing audio in testTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4


In [1]:
from src.embed import *

In [9]:
meta = pd.read_csv(f"outputs/test/metadata.csv", lineterminator="\n")#.iloc[:300]
    
dim = next(model.parameters()).shape[1]
print(f"embedding dimensionality = {dim}")
chunk_size = min(len(meta), 7)
    
embs = np.zeros((meta.shape[0], dim))
cur_ind = 0
# # for i, txt in tqdm(enumerate(meta.pretty_text), total=meta.shape[0]):
chunks = np.array_split(list(meta.pretty_text), int(len(meta)/chunk_size))
# # for txts in tqdm(chunks, total=len(chunks)):


embedding dimensionality = 768


In [10]:
chunks

[array(['Title: أقوى منشط جذور طبيعي 100% وعن تجربة @youtubecreators #YuoTubeHighFive\n\nChannel: بالمختصر المفيد - مسعد نصار\n\nDescription: يتناول المقطع كيفية تحضير محلول يعتبر من أقوي المحاليل الطبيعية المنشطة لنمو الجذور والشعيرات الجذرية من مكونات طبيعية ١٠٠% .\nمكوناته : عشبة العرقسوس والشوفان وجل صبار الالوفيرا .\n---------\nمع توضيح ما يحويه من عناصر وهرمونات نمو ومواد طبيعية فعالة قابلة للامتصاص .\nويتناول تجربة حقيقية فعلية لهذا المحلول وكيفية إعداده واستخدامه ونتيجته حتى تتحقق المصداقية والشفافية\n#بالمختصر_المفيد\n#مسعد_نصار \n#الزراعة_المنزلية \n#الزراعة_العضوية\n#اكسبلور \n#هرمونات _تجذير\n#منشط_جذور\n#تجذير_الشتلات\n#تسميد\n#نباتات \n#زراعة \n#زراعة_الاسطح \n\n#farming \n#gardening \n #farm\n#explore\n#plants \n#fertilization\n#roof_cultivation\n#organic_fertilizer\n#home_farming\n#root_stimulant\n#rooting_hormone\n #YuoTubeHighFive\n\n',
        'Title: تعفن الجذور ماله علاقة بكمية الري؟؟\n\nChannel: Ola Bawarshi - علا بوارشي\n\nDescription: كمية المي يلي بنسقيها لنبات

---

# making sure files are small